In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[4],3b
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,0
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[5, 4]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  76.722160                    1  54.493066
0.000750        2  40.961180                    2  41.525478
0.001052        3  33.041424                    3  35.820237
0.001476        4  26.177150                    4  29.287412
0.002070        5  20.453587                    5  22.420878
0.002904        6  15.887011                    6  16.167919
0.004074        7  12.355441                    7  11.339904
0.005714        8   9.670387                    8   8.313265
0.008015        9   7.632119                    9   6.630614
0.011243       10   6.059189                   10   5.575373
0.015771       11   4.800884                   11   4.583296
0.022122       12   3.733239                   12   3.497241
0.031031       13   2.741506                   13   2.365678
0.043528       14   1.535104                   14   1.084163
0.061057       15   0.621270                   15   0.254321
0.085645       16   1.009105                   16   0.808563
0.120136       17   1.978014                   17   1.947530
0.168516       18   2.672370                   18   2.797531
0.236378       19   3.351253                   19   3.530244
0.331549       20   4.134225                   20   4.169617
0.465100       21   5.128997                   21   4.930275
0.652400       22   6.668315                   22   6.395361
0.915100       23   8.412267                   23   8.259775
1.283650       24   8.111907                   24   8.022097
1.800600       25   6.297593                   25   6.104184
2.525700       26   4.914451                   26   4.619267
3.542800       27   3.934145                   27   3.883225
4.969550       28   3.201959                   28   3.336434
6.970850       29   2.668698                   29   2.762403
9.778100       30   2.244096                   30   2.256906
13.715850      31   1.822880                   31   1.847579
19.239350      32   1.385437                   32   1.349680
26.987250      33   1.051152                   33   1.013075
37.855300      34   0.824001                   34   0.878007
53.100050      35   0.576221                   35   0.632129
73.887500      36   0.318069                   36   0.302485
97.662500      37   0.177026                   37   0.150638
121.437500     38   0.131797                   38   0.137450
145.212500     39   0.032605                   39   0.057518
168.987500     40  -0.100428                   40  -0.075853
192.762500     41  -0.096020                   41  -0.080889
216.537500     42   0.023673                   42   0.026030
240.312500     43   0.075846                   43   0.068992
264.087500     44   0.091997                   44   0.080934
287.862500     45   0.098112                   45   0.085719
311.637500     46   0.099724                   46   0.085334
335.412500     47   0.098896                   47   0.085435
359.187500     48   0.096725                   48   0.083394
382.962500     49   0.093992                   49   0.081971
406.737500     50   0.091034                   50   0.081379
430.512500     51   0.087524                   51   0.080779
454.287500     52   0.082854                   52   0.077848
478.062500     53   0.076838                   53   0.074488
501.837500     54   0.070158                   54   0.069393
525.612500     55   0.063739                   55   0.064401
549.387500     56   0.058049                   56   0.059402
573.162500     57   0.053072                   57   0.054610
596.937500     58   0.048726                   58   0.050104
620.712500     59   0.044924                   59   0.045948
644.487500     60   0.041549                   60   0.042133
668.262500     61   0.038595                   61   0.038711
692.037500     62   0.036114                   62   0.034709
715.812500     63   0.034227                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -17.779314   0.000000 -17.779314 -17.469121 -1.304204e-07   
0.000624    2     -17.779655   0.006016 -17.773639 -17.469496  4.406678e-03   
0.000876    3     -17.779811   0.007392 -17.772418 -17.469654  5.801405e-03   
0.001229    4     -17.780008   0.008970 -17.771038 -17.469858  7.502311e-03   
0.001723    5     -17.780256   0.010752 -17.769504 -17.470121  9.481842e-03   
0.002417    6     -17.780569   0.012747 -17.767822 -17.470461  1.166513e-02   
0.003391    7     -17.780969   0.014979 -17.765990 -17.470901  1.397044e-02   
0.004757    8     -17.781482   0.017490 -17.763991 -17.471478  1.638179e-02   
0.006672    9     -17.782148   0.020351 -17.761797 -17.472236  1.902605e-02   
0.009359    10    -17.783026   0.023658 -17.759368 -17.473244  2.214477e-02   
0.013128    11    -17.784201   0.027538 -17.756663 -17.474598  2.598774e-02   
0.018415    12    -17.785805   0.032149 -17.753656 -17.476448  3.070923e-02   
0.025830    13    -17.788054   0.037677 -17.750377 -17.479034  3.636732e-02   
0.036232    14    -17.791335   0.044337 -17.746999 -17.482762  4.301068e-02   
0.050823    15    -17.796754   0.052409 -17.744346 -17.488727  5.084976e-02   
0.071291    16    -17.806083   0.063244 -17.742839 -17.498626  6.136486e-02   
0.100000    17    -17.819859   0.080451 -17.739407 -17.512990  7.847975e-02   
0.140271    18    -17.837769   0.107797 -17.729972 -17.531504  1.062851e-01   
0.196760    19    -17.860581   0.148492 -17.712089 -17.554780  1.482823e-01   
0.275997    20    -17.888973   0.208339 -17.680634 -17.583260  2.099014e-01   
0.387100    21    -17.922659   0.296436 -17.626224 -17.616436  2.979596e-01   
0.543100    22    -17.958270   0.426827 -17.531443 -17.650842  4.234824e-01   
0.761700    23    -17.978681   0.619912 -17.358769 -17.669877  6.081405e-01   
1.068500    24    -17.941817   0.888772 -17.053045 -17.633200  8.716766e-01   
1.498800    25    -17.834572   1.195008 -16.639564 -17.529646  1.177069e+00   
2.102400    26    -17.688006   1.498724 -16.189282 -17.388074  1.471996e+00   
2.949000    27    -17.512420   1.815988 -15.696432 -17.217726  1.764942e+00   
4.136600    28    -17.309235   2.166257 -15.142978 -17.018171  2.111735e+00   
5.802500    29    -17.080558   2.569448 -14.511111 -16.792239  2.544274e+00   
8.139200    30    -16.822858   3.050440 -13.772418 -16.534358  3.051102e+00   
11.417000   31    -16.527399   3.626315 -12.901084 -16.235366  3.628510e+00   
16.014700   32    -16.200522   4.292234 -11.908288 -15.901215  4.300711e+00   
22.464000   33    -15.884825   5.034963 -10.849862 -15.578717  5.009428e+00   
31.510500   34    -15.603034   5.879610  -9.723424 -15.290413  5.806870e+00   
44.200100   35    -15.321586   6.836779  -8.484808 -15.003930  6.840322e+00   
62.000000   36    -15.069403   7.799571  -7.269832 -14.749148  7.918538e+00   
85.775000   37    -14.997598   8.623551  -6.374047 -14.678438  8.699811e+00   
109.550000  38    -15.099057   9.223570  -5.875487 -14.788024  9.233684e+00   
133.325000  39    -15.263683   9.759378  -5.504305 -14.964812  9.797616e+00   
157.100000  40    -15.604738  10.192260  -5.412478 -15.322769  1.031758e+01   
180.875000  41    -16.352852  10.657536  -5.695316 -16.097119  1.087828e+01   
204.650000  42    -17.440965  11.475226  -5.965739 -17.215145  1.176847e+01   
228.425000  43    -18.588967  12.689898  -5.899069 -18.391579  1.301822e+01   
252.200000  44    -19.724622  14.039158  -5.685464 -19.549890  1.437086e+01   
275.975000  45    -20.842292  15.415922  -5.426370 -20.686905  1.573583e+01   
299.750000  46    -21.940872  16.790817  -5.150056 -21.801756  1.709212e+01   
323.525000  47    -23.019646  18.150446  -4.869201 -22.894569  1.842529e+01   
347.300000  48    -24.077740  19.487061  -4.590679 -23.964251  1.973561e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')